In [ ]:
project_list = [
    'assertj-assertions-generator',
    'commons-cli',
    'commons-csv',
    'commons-codec',
    'delight-nashorn-sandbox',
    'empire-db',
    'jimfs',
    # 'handlebars.java',
    # 'httpcore',
    # 'riptide',

    # 'commons-net',
    # 'commons-collections',
    # 'commons-net',
    # 'empire-db',
    # 'guava',
    # 'java-design-patterns',
    # 'jooby',
    # 'maven-dependency-plugin',
    # 'maven-shade-plugin',
    # 'sling-org-apache-sling-auth-core',
    # 'stream-lib'
]
seed_list = [
    0,
    42,
    123,
    216,
    1202,
    1999,
    2002,
    2024,
    31415,
    99999,
    'default',
    'fastest'
]
round_number = 6
seed_number = len(seed_list)
parsed_path = 'controlled_parsed_data'
analyzed_path = 'controlled_analyzed_data'

## 1 Get total running time

In [ ]:
import pandas as pd
import numpy as np
import json
from scipy.stats import ttest_ind, mannwhitneyu


def get_info(file_path):
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as f:
        id_runtimes_dict = json.load(f)
    return id_runtimes_dict


choice = 'more_projects'
significant_df = pd.DataFrame(None, columns=['project', 'seed1', 'seed2', 'T-test', 'U-test'])
for project in project_list:
    columns = ['seed'] + [f'round{i}' for i in range(round_number)]
    columns += ['avg.', '/avg. default', '/avg. fastest']
    runtime_df = pd.DataFrame(None, columns=columns)
    seed_runtimes_dict = {}
    for seed in seed_list:
        runtime_list = np.zeros(round_number, dtype=int)
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)
        per_id_runtimes_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        for mut_id in non_flaky_id_list:
            runtime_list += np.array(per_id_runtimes_dict[mut_id])
        seed_runtimes_dict[seed] = runtime_list

    avg_default = int(round(np.mean(seed_runtimes_dict['default'])))
    avg_fastest = int(round(np.mean(seed_runtimes_dict['fastest'])))
    for i in range(seed_number):
        seed1 = seed_list[i]
        runtimes1 = seed_runtimes_dict[seed1]
        avg_runtime = int(round(np.mean(runtimes1)))
        ratio_vs_default = round(avg_runtime / avg_default, 3)
        ratio_vs_fastest = round(avg_runtime / avg_fastest, 3)
        infos = [seed1] + list(runtimes1) + [f'{avg_runtime}', f'{ratio_vs_default}', f'{ratio_vs_fastest}']
        runtime_df.loc[len(runtime_df.index)] = infos
        for j in range(seed_number):
            if j > i:
                seed2 = seed_list[j]
                runtimes2 = seed_runtimes_dict[seed2]
                t_stat, t_p_value = ttest_ind(runtimes1, runtimes2)
                u_stat, u_p_value = mannwhitneyu(runtimes1, runtimes2)
                significant_df.loc[len(significant_df.index)] = [project, seed1, seed2, f'{t_p_value:.3f}', f'{u_p_value:.3f}']
    runtime_df.to_csv(f'{analyzed_path}/{choice}/total_runtime/{project}.csv', sep=',', header=True, index=False)
significant_df.to_csv(f'{analyzed_path}/{choice}/total_runtime/significant_results.csv', sep=',', header=True, index=False)

## 2 Calculate T-test, U-test

In [ ]:
import pandas as pd
import warnings
import json
from scipy.stats import ttest_ind, mannwhitneyu
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'
warnings.filterwarnings('ignore', category=RuntimeWarning, message='Precision loss occurred in moment calculation.')


def get_info(project, seed):
    path = f'{parsed_path}/{choice}/{project}_{seed}'
    with open(f'{path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


def to_be_calculable(runtime_list):
    output_list = []
    for runtime in runtime_list:
        if runtime == TIMED_OUT:
            output_list.append(6000)
        else:
            output_list.append(runtime)
    return output_list


for project in project_list:
    seed_mutants_dict = {}
    for seed in seed_list:
        mutantId_mutantTuple_dict, mutantId_runtimeList_dict = get_info(project, seed)
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        seed_mutants_dict[seed] = {}
        for mutant_id in non_flaky_list:
            mutant_pref = tuple(mutantId_mutantTuple_dict[mutant_id][:5])
            seed_mutants_dict[seed][mutant_pref] = to_be_calculable(mutantId_runtimeList_dict[mutant_id])
    for i in range(seed_number):
        i_seed = seed_list[i]
        i_mutants_dict = seed_mutants_dict[i_seed]
        j = i + 1
        while j < seed_number:
            df = pd.DataFrame(None, columns=['mutant', 'T-test', 'U-test'])
            j_seed = seed_list[j]
            j_mutants_dict = seed_mutants_dict[j_seed]
            significant_number = 0
            for pref, i_runtime_list in i_mutants_dict.items():
                j_runtime_list = j_mutants_dict[pref]
                t_stat, t_p_value = ttest_ind(i_runtime_list, j_runtime_list)
                u_stat, u_p_value = mannwhitneyu(i_runtime_list, j_runtime_list)
                if t_p_value < 0.05 or u_p_value < 0.05:
                    significant_number += 1
                df.loc[len(df.index)] = [pref, t_p_value, u_p_value]
            # df.to_csv(f'{analyzed_path}/{choice}/significance_detection/non-flaky/{project}_{i_seed}_{j_seed}.csv', sep=',', header=True, index=False)
            j += 1
            print(f'{project} with ({i_seed}, {j_seed}): {significant_number}/{len(non_flaky_list)}')

## 3. Calculate proportions

In [ ]:
import pandas as pd
import random
import json
from pitest_log_parser import mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict, mutantId_runtimeList_dict


for project in project_list:
    total_flaky_tests = set()
    for seed in seed_list:
        flaky_chaotic_order_mutant_list = []
        total_num = 0
        non_flaky_num = 0
        flaky_chaotic_order_num = 0
        uniqueId_mutantIds_dict = {}
        mutantId_mutantTuple_dict, mutantId_testsInOrder_dict, mutantId_runtimeList_dict = get_info(f'parsed_data/{choice}/{project}_{seed}')
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            unique_id = tuple(mutant_tuple[:5])
            test_list = mutantId_testsInOrder_dict[mutant_id]
            if unique_id not in uniqueId_mutantIds_dict:
                uniqueId_mutantIds_dict[unique_id] = set()
            uniqueId_mutantIds_dict[unique_id].add(mutant_id)
        total_num = len(uniqueId_mutantIds_dict)
        for unique_id, mutant_ids in uniqueId_mutantIds_dict.items():
            if len(mutant_ids) == 1:
                non_flaky_num += 1
            else:
                test_intersection_set = None
                for mutant_id in mutant_ids:
                    per_test_set = set(mutantId_testsInOrder_dict[mutant_id])
                    if test_intersection_set is None:
                        test_intersection_set = per_test_set
                    else:
                        test_intersection_set &= per_test_set
                is_same = True
                flaky_tests = set()
                for mutant_id in mutant_ids:
                    per_test_set = set(mutantId_testsInOrder_dict[mutant_id])
                    if len(per_test_set) == len(test_intersection_set):
                        continue
                    flaky_tests |= (per_test_set - test_intersection_set)
                if len(flaky_tests) == 0:
                    flaky_chaotic_order_num += 1
                    flaky_chaotic_order_mutant_list.append(random.choice(list(mutant_ids)))
                total_flaky_tests |= flaky_tests
        # with open(f'analyzed_data/{choice}/mutant_list/chaotic_order/{project}_{seed}.json', 'w') as file:
        #     json.dump(flaky_chaotic_order_mutant_list, file, indent=4)
    
        print(f'{project} with {seed}:')
        print(f'Total mutants: {total_num}')
        print(f'Non-flaky mutants: {non_flaky_num} ({non_flaky_num / total_num:.4f})')
        print(f'Flaky mutants due to chaotic test list: {flaky_chaotic_order_num} ({flaky_chaotic_order_num / total_num:.4f})')

## 4. Create guiding files

In [ ]:
import random
import json
from pitest_log_parser import project_junitVersion_dict, mutant_choice, test_choice, TIMED_OUT
random_mutant = False
random_test = True
choice = f'{mutant_choice[random_mutant]}_{test_choice[random_test]}'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict


def mutant_to_json(mutant, test_list, junit_version):
    def test_sort(test):
        return (test['name'])
    
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    if random_test:
        random.shuffle(test_order)
    else:
        test_order.sort(key=test_sort)

    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'filename': mutant[5],
        'block': f'[{mutant[6]}]',
        'lineNumber': int(mutant[7]),
        'description': mutant[8],
        'testsInOrder': test_order
    }


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )


parent_path = f'analyzed_data/{choice}/mutant_list'
for project in project_list:
    junit_version = project_junitVersion_dict[project]
    # get the mutant intersection
    id_set = set()
    for seed in seed_list:
        mutantId_mutantTuple_dict, _ = get_info(f'parsed_data/{choice}/{project}_{seed}')
        with open(f'{parent_path}/chaotic_order/{project}_{seed}.json', 'r') as file:
            chaotic_order_list = json.load(file)
        per_id_set = set()
        for mutant_id in chaotic_order_list:
            per_id_set.add(tuple(mutantId_mutantTuple_dict[mutant_id][:5]))
        if len(id_set) == 0:
            id_set |= per_id_set
        else:
            id_set &= per_id_set
    # create guiding mutant list
    for seed in seed_list:
        with open(f'{parent_path}/non-flaky/{project}_{seed}.json', 'r') as file:
            non_flaky_list = json.load(file)
        mutantId_mutantTuple_dict, mutantId_testsInOrder_dict = get_info(f'parsed_data/{choice}/{project}_{seed}')
        output_list = []
        id_visited_dict = {item: True for item in id_set}
        for mutant_id, mutant_tuple in mutantId_mutantTuple_dict.items():
            id_tuple = tuple(mutant_tuple[:5])
            if mutant_id in non_flaky_list or (id_tuple in id_set and id_visited_dict[id_tuple]):
                if id_tuple in id_set:
                    id_visited_dict[id_tuple] = False
                output_list.append(mutant_to_json(mutant=mutant_tuple,
                                                  test_list=mutantId_testsInOrder_dict[mutant_id],
                                                  junit_version=junit_version))
        if random_mutant:
            random.shuffle(output_list)
        else:
            output_list.sort(key=mutant_sort)
        with open(f'analyzed_data/{choice}/guiding_files/{project}_{seed}.json', 'w') as f:
            f.write(json.dumps(output_list, indent=4))

## 5. Choose fastest test order to guide

In [ ]:
import json
import numpy as np
from pitest_log_parser import project_junitVersion_dict
seeds = [0, 42, 123, 216, 1202, 1999, 2002, 2024, 31415, 99999]
choice = 'more_projects'

def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict, mutantId_testsInOrder_dict
    

def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )
    

def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'testsInOrder': test_order
    }


for project in ['commons-cli']:
    junit_version = project_junitVersion_dict[project]
    mutation_runtimes_dict = {}
    mutation_tests_dict = {}
    is_recorded = False
    for seed in seeds:
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_list = json.load(f)
        id_tuple_dict, id_runtimes_dict, id_tests_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        for mut_id in non_flaky_list:
            mut = tuple(id_tuple_dict[mut_id])
            runtime_list = id_runtimes_dict[mut_id]
            runtime = round(np.mean(runtime_list))
            if is_recorded:
                mutation_runtimes_dict[mut].append(runtime)
                mutation_tests_dict[mut].append(id_tests_dict[mut_id])
            else:
                mutation_runtimes_dict[mut] = [runtime]
                mutation_tests_dict[mut] = [id_tests_dict[mut_id]]
        is_recorded = True
    output_list = []
    for mut, runtimes in mutation_runtimes_dict.items():
        tests = mutation_tests_dict[mut]
        mini = 1000000
        mini_index = -1
        for i in range(len(seeds)):
            if runtimes[i] < mini:
                mini = runtimes[i]
                mini_index = i
        output_list.append(mutant_to_json(mutant=mut,
                                          test_list=tests[mini_index],
                                          junit_version=junit_version))
    output_list.sort(key=mutant_sort)
    with open(f'{analyzed_path}/{choice}/guiding_files/{project}_fastest.json', 'w') as f:
        f.write(json.dumps(output_list, indent=4))

## 6. Generate input files based on single default result

In [ ]:
import random
import json
from pitest_log_parser import project_junitVersion_dict


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_testsInOrder.json', 'r') as file:
        mutantId_testsInOrder_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_testsInOrder_dict


def mutant_to_json(mutant, test_list, junit_version):
    test_order = []
    if junit_version == 'junit4':
        for test in test_list:
            paren_index = test.find('(')
            for i in range(paren_index, 0, -1):
                if test[i] == '.': break
            test_order.append({
                'definingClass': test[:i],
                'name': test
            })
    else:
        for test in test_list:
            class_end_index = test.find('[')
            test_order.append({
                'definingClass': test[:class_end_index-1],
                'name': test
            })
    # random.shuffle(test_order)
    return {
        'id': {
            'location': {
                'clazz': mutant[0],
                'method': mutant[1],
                'methodDesc': mutant[2]
            },
            'indexes': f'[{mutant[3]}]',
            'mutator': mutant[4]
        },
        'filename': mutant[5],
        'block': f'[{mutant[6]}]',
        'lineNumber': int(mutant[7]),
        'description': mutant[8],
        'testsInOrder': test_order
    }


def mutant_sort(mutant):
    return (
        mutant['id']['location']['clazz'],
        mutant['id']['location']['method'],
        mutant['id']['location']['methodDesc'],
        mutant['id']['indexes'],
        mutant['id']['mutator']
    )


for project in project_list:
    junit_version = project_junitVersion_dict[project]
    id_tuple_dict, id_tests_dict = get_info(f'parsed_data/default_version/{project}')
    for seed in seed_list:
        # random.seed(seed)
        seed = 'default'
        output_list = []
        for mut_id, mut_tup in id_tuple_dict.items():
            output_list.append(mutant_to_json(mutant=mut_tup,
                                              test_list=id_tests_dict[mut_id],
                                              junit_version=junit_version))
        output_list.sort(key=mutant_sort)
        with open(f'analyzed_data/default_version/guiding_files/{project}_{seed}.json', 'w') as f:
            f.write(json.dumps(output_list, indent=4))

## 7. Analyze the order of mutants

In [6]:
import json
from flakiness_filter import clazz_index


choice = 'more_projects'
print('For each clazz (each process):')
for project in project_list:
    clazz_orders_dict = dict()
    for seed in seed_list:
        cur_clazz_orders_dict = dict()
        with open(f'{parsed_path}/{choice}/{project}_{seed}/mutantId_mutantTuple.json', 'r') as f:
            id_tuple_dict = json.load(f)
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)

        for mut_id in non_flaky_id_list:
                mut_tup = id_tuple_dict[mut_id]
                clazz = mut_tup[clazz_index]
                if clazz in cur_clazz_orders_dict:
                    cur_clazz_orders_dict[clazz].append(mut_tup)
                else:
                    cur_clazz_orders_dict[clazz] = [mut_tup]
        diff_cnt = 0
        if len(clazz_orders_dict) == 0:
            clazz_orders_dict = cur_clazz_orders_dict
        else:
            for clazz, mutant_order in clazz_orders_dict.items():
                if mutant_order != cur_clazz_orders_dict[clazz]:
                    diff_cnt += 1
        if diff_cnt > 0:
            print(f'The diff cnt of {project} with {seed}: {diff_cnt}')

For each clazz (each process):


## 8. Average runtime (per mutant) table

In [ ]:
import pandas as pd
import numpy as np


def get_info(file_path):
    with open(f'{file_path}/mutantId_mutantTuple.json', 'r') as file:
        mutantId_mutantTuple_dict = json.load(file)
    with open(f'{file_path}/mutantId_runtimeList.json', 'r') as file:
        mutantId_runtimeList_dict = json.load(file)
    return mutantId_mutantTuple_dict, mutantId_runtimeList_dict


choice = 'more_projects'
cols = ['clazz', 'method', 'methodDesc', 'indexes', 'mutator'] + [f'{s}' for s in seed_list]
for project in project_list:
    df = pd.DataFrame(None, columns=cols)
    mutant_runtimes_dict = dict()
    is_recorded = False
    for seed in seed_list:
        id_tuple_dict, id_runtimes_dict = get_info(f'{parsed_path}/{choice}/{project}_{seed}')
        with open(f'{analyzed_path}/{choice}/mutant_list/non-flaky/{project}_{seed}.json', 'r') as f:
            non_flaky_id_list = json.load(f)
        for mut_id in non_flaky_id_list:
            mut_tup = tuple(id_tuple_dict[mut_id])
            avg_runtime = round(np.mean(id_runtimes_dict[mut_id]))
            if is_recorded:
                mutant_runtimes_dict[mut_tup].append(avg_runtime)
            else:
                mutant_runtimes_dict[mut_tup] = [avg_runtime]
        is_recorded = True
    for mut, avg_runtimes in mutant_runtimes_dict.items():
        df.loc[len(df.index)] = list(mut) + avg_runtimes
    df.to_csv(f'{analyzed_path}/{choice}/runtime_per_mutant/{project}.csv', sep=',', header=True, index=False)